In [4]:
import pandas as pd

import collections

import numpy as np

import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Dropout, LSTM
from tensorflow.keras.layers import Embedding
from keras.optimizers import Adam 
from keras.losses import sparse_categorical_crossentropy
import sklearn
from sklearn.model_selection import train_test_split
import pickle


Bringing in the Texx

In [93]:
en_path = 'small_vocab_en.txt'
fr_path = 'small_vocab_fr.txt'


In [94]:
with open(en_path, 'r') as file:
    en_sentences = file.read()
    en_sentences = en_sentences.split('\n')
    en_sentences = [sentence.strip() for sentence in en_sentences if sentence.strip()]

with open(fr_path, 'r') as file:
    fr_sentences = file.read()
    fr_sentences = fr_sentences.split('\n')
    fr_sentences = [sentence.strip() for sentence in fr_sentences if sentence.strip()]

print("text uploaded")

text uploaded


In [95]:
en_sentences


['new jersey is sometimes quiet during autumn , and it is snowy in april .',
 'hello my name is Pavle .',
 'the united states is usually chilly during july , and it is usually freezing in november .',
 'california is usually quiet during march , and it is usually hot in june .',
 'the united states is sometimes mild during june , and it is cold in september .',
 'your least liked fruit is the grape , but my least liked is the apple .',
 'his favorite fruit is the orange , but my favorite is the grape .',
 'paris is relaxing during december , but it is usually chilly in july .',
 'new jersey is busy during spring , and it is never hot in march .',
 'our least liked fruit is the lemon , but my least liked is the grape .',
 'the united states is sometimes busy during january , and it is sometimes warm in november .',
 'the lime is her least liked fruit , but the banana is my least liked .',
 'he saw a old yellow truck .',
 'india is rainy during june , and it is sometimes warm in november

In [96]:
print(len(en_sentences))

137861


In [97]:
fr_sentences

["new jersey est parfois calme pendant l' automne , et il est neigeux en avril .",
 "Bonjour, je m'appelle .",
 'les Ã©tats-unis est gÃ©nÃ©ralement froid en juillet , et il gÃ¨le habituellement en novembre .',
 'california est gÃ©nÃ©ralement calme en mars , et il est gÃ©nÃ©ralement chaud en juin .',
 'les Ã©tats-unis est parfois lÃ©gÃ¨re en juin , et il fait froid en septembre .',
 'votre moins aimÃ© fruit est le raisin , mais mon moins aimÃ© est la pomme .',
 "son fruit prÃ©fÃ©rÃ© est l'orange , mais mon prÃ©fÃ©rÃ© est le raisin .",
 'paris est relaxant en dÃ©cembre , mais il est gÃ©nÃ©ralement froid en juillet .',
 'new jersey est occupÃ© au printemps , et il est jamais chaude en mars .',
 'notre fruit est moins aimÃ© le citron , mais mon moins aimÃ© est le raisin .',
 'les Ã©tats-unis est parfois occupÃ© en janvier , et il est parfois chaud en novembre .',
 'la chaux est son moins aimÃ© des fruits , mais la banane est mon moins aimÃ©.',
 'il a vu un vieux camion jaune .',
 'inde est

In [98]:
print(len(fr_sentences))

137861


Tokenize Function

In [11]:
def tokenize(data):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(data)
    return tokenizer.texts_to_sequences(data), tokenizer

Padding Sentences Function

In [12]:
def pad(data, length=None):
    return pad_sequences(data, maxlen = length, padding = 'post')

Preprocessing Function

In [13]:
def preprocess(lang1, lang2):
    preprocess_lang1, lang1_tk = tokenize(lang1)
    preprocess_lang2, lang2_tk = tokenize(lang2)

    preprocess_lang1 = pad(preprocess_lang1)
    preprocess_lang2 = pad(preprocess_lang2)

    preprocess_lang2 = preprocess_lang2.reshape(*preprocess_lang2.shape, 1)
    return preprocess_lang1, preprocess_lang2, lang1_tk, lang2_tk

In [107]:
all_sentences = en_sentences + fr_sentences

# Tokenize and pad the sentences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_sentences)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_sentences)

# Tokenize English sentences
en_tokenizer = Tokenizer()
en_tokenizer.word_index = tokenizer.word_index
preproc_en_sentences = en_tokenizer.texts_to_sequences(en_sentences)
preproc_en_sentences = pad_sequences(preproc_en_sentences, maxlen=15, padding='post')

# Tokenize French sentences
fr_tokenizer = Tokenizer()
fr_tokenizer.word_index = tokenizer.word_index
preproc_fr_sentences = fr_tokenizer.texts_to_sequences(fr_sentences)
preproc_fr_sentences = pad_sequences(preproc_fr_sentences, maxlen=21, padding='post')


In [106]:
tokenizer_path = 'tokenizer.json'
tokenizer_json = tokenizer.to_json()
with open(tokenizer_path, 'w', encoding='utf-8') as f:
    f.write(tokenizer_json)

Preprocessing Data

In [14]:
#preproc_en_sentences, preproc_fr_sentences, en_tokenizer, fr_tokenizer = preprocess(en_sentences, fr_sentences)

In [108]:
tokenizer_path = 'fr_tokenizer.json'
tokenizer_json = fr_tokenizer.to_json()
with open(tokenizer_path, 'w', encoding='utf-8') as f:
    f.write(tokenizer_json)

In [110]:
tokenizer_path = 'en_tokenizer.json'
tokenizer_json = en_tokenizer.to_json()
with open(tokenizer_path, 'w', encoding='utf-8') as f:
    f.write(tokenizer_json)

In [18]:
print(en_sentences[:5], preproc_en_sentences[:5], len(preproc_en_sentences[0]))

['new jersey is sometimes quiet during autumn , and it is snowy in april .', 'the united states is usually chilly during july , and it is usually freezing in november .', 'california is usually quiet during march , and it is usually hot in june .', 'the united states is sometimes mild during june , and it is cold in september .', 'your least liked fruit is the grape , but my least liked is the apple .'] [[17 23  1  8 67  4 39  7  3  1 55  2 44  0  0]
 [ 5 20 21  1  9 62  4 43  7  3  1  9 51  2 45]
 [22  1  9 67  4 38  7  3  1  9 68  2 34  0  0]
 [ 5 20 21  1  8 64  4 34  7  3  1 57  2 42  0]
 [29 12 16 13  1  5 82  6 30 12 16  1  5 83  0]] 15


In [19]:
print(preproc_en_sentences[0])

[17 23  1  8 67  4 39  7  3  1 55  2 44  0  0]


In [20]:
max_en_sequence_length = len(preproc_en_sentences[0])
max_fr_sequence_length = len(preproc_fr_sentences[0])

en_vocab_size = len(en_tokenizer.word_index)
fr_vocab_size = len(fr_tokenizer.word_index)


In [21]:
print(f"Max en sentences length: {max_en_sequence_length}")
print(f"Max fr sentences length: {max_fr_sequence_length}")

Max en sentences length: 15
Max fr sentences length: 21


In [22]:
print(f"En vocab size: {en_vocab_size}")
print(f"Fr vocab size: {fr_vocab_size}")

En vocab size: 199
Fr vocab size: 345


Function to Convert From Id's to Text

In [23]:
def id_to_text(logits, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return " ".join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

Building the Network(Bidirectional RNN)

In [70]:
def model(input_shape,output_sequence_length, english_vocab_size, french_vocab_size):
    learning_rate = 0.003
    
    # Build the layers    
    model = Sequential()
    # Embedding
    model.add(Embedding(english_vocab_size, 128, input_length=input_shape[1],
                         input_shape=input_shape[1:]))
    # Encoder
    model.add(Bidirectional(GRU(128)))
    model.add(RepeatVector(output_sequence_length))
    # Decoder
    model.add(Bidirectional(GRU(128, return_sequences=True)))
    model.add(TimeDistributed(Dense(512, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model


In [75]:
tmp_lang1 = pad(preproc_en_sentences, preproc_fr_sentences.shape[1])
tmp_lang1 = tmp_lang1.reshape((-1, preproc_fr_sentences.shape[-2]))
print(tmp_lang1.shape, preproc_fr_sentences.shape)

(137860, 21) (137860, 21, 1)


In [76]:
X_train, X_test, y_train, y_test = train_test_split(tmp_lang1, preproc_fr_sentences, test_size=0.1, random_state=42)

In [77]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(124074, 21) (124074, 21, 1)
(13786, 21) (13786, 21, 1)


In [78]:
model = model(X_train.shape, y_train.shape[1], en_vocab_size + 1, fr_vocab_size + 1)

In [80]:
model.fit(X_train, y_train, batch_size= 1024, epochs = 25, validation_split=0.2)
model.summary()

Epoch 1/25
97/97 [==============================] - 246s 3s/step - loss: 2.2177 - accuracy: 0.4971 - val_loss: 1.5992 - val_accuracy: 0.5889
Epoch 2/25
97/97 [==============================] - 232s 2s/step - loss: 1.4652 - accuracy: 0.6125 - val_loss: 1.1984 - val_accuracy: 0.6684
Epoch 3/25
97/97 [==============================] - 240s 2s/step - loss: 1.2233 - accuracy: 0.6607 - val_loss: 1.0433 - val_accuracy: 0.7021
Epoch 4/25
97/97 [==============================] - 257s 3s/step - loss: 1.0569 - accuracy: 0.6979 - val_loss: 0.9246 - val_accuracy: 0.7299
Epoch 5/25
97/97 [==============================] - 237s 2s/step - loss: 0.9476 - accuracy: 0.7221 - val_loss: 0.8334 - val_accuracy: 0.7471
Epoch 6/25
97/97 [==============================] - 236s 2s/step - loss: 0.8862 - accuracy: 0.7365 - val_loss: 0.7336 - val_accuracy: 0.7746
Epoch 7/25
97/97 [==============================] - 362s 4s/step - loss: 0.7495 - accuracy: 0.7706 - val_loss: 0.6313 - val_accuracy: 0.8059
Epoch 8/25
97

In [81]:
model.save("translator")

INFO:tensorflow:Assets written to: translator\assets


INFO:tensorflow:Assets written to: translator\assets


In [82]:
model = tf.keras.models.load_model('translator')
print(id_to_text(model.predict(tmp_lang1[:1])[0], fr_tokenizer))

1/1 [==============================] - 1s 1s/step
new jersey est parfois calme pendant l' automne il il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


In [89]:
print(tmp_lang1[:1])

[[17 23  1  8 67  4 39  7  3  1 55  2 44  0  0  0  0  0  0  0  0]]


In [83]:
predictions = model.predict(X_test)

431/431 [==============================] - 38s 87ms/step


In [84]:
evaluation = model.evaluate(X_test, y_test)

431/431 [==============================] - 41s 92ms/step - loss: 0.1289 - accuracy: 0.9614


In [85]:
print("Predictions:", predictions)


Predictions: [[[5.8258278e-28 4.2125595e-14 2.1124401e-35 ... 3.8754090e-28
   1.5160414e-27 8.0093085e-26]
  [9.2722405e-28 1.0000000e+00 0.0000000e+00 ... 4.8874836e-30
   1.9643407e-28 2.4378273e-27]
  [7.3909278e-24 2.4685205e-06 1.3355569e-35 ... 3.3155758e-27
   1.8773270e-26 4.9583769e-26]
  ...
  [1.0000000e+00 6.0330819e-27 1.0734536e-22 ... 0.0000000e+00
   0.0000000e+00 0.0000000e+00]
  [1.0000000e+00 1.6143386e-27 3.0325312e-23 ... 0.0000000e+00
   0.0000000e+00 0.0000000e+00]
  [1.0000000e+00 6.4520490e-25 1.8892686e-20 ... 0.0000000e+00
   0.0000000e+00 0.0000000e+00]]

 [[7.8404413e-22 0.0000000e+00 0.0000000e+00 ... 3.7874738e-21
   2.6341365e-21 2.1864266e-21]
  [2.6732695e-14 2.5731509e-24 3.4299398e-30 ... 1.8681696e-14
   1.9832055e-14 1.8067421e-14]
  [1.0215327e-09 4.8088053e-13 2.6069594e-16 ... 1.3396698e-11
   1.5380440e-11 3.5746340e-11]
  ...
  [1.0000000e+00 3.3519698e-24 2.2972035e-21 ... 3.1939989e-38
   0.0000000e+00 0.0000000e+00]
  [1.0000000e+00 6.3786

In [86]:
print("Evaluation:", evaluation)

Evaluation: [0.12893228232860565, 0.9613548517227173]
